# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import plotter
import utils
import pathing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gpd
import pandas as pd

import osmnx as ox
import networkx as nx
import geopandas as gpd
import shapely
import shapely.geometry
import folium
import scipy
import networkx


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Main

In [5]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

In [6]:
central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)

In [7]:
def get_node(x, y):
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)

In [8]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times):
    total_diff = 0
    total_diff_percentage = 0

    for destination, benchmark in benchmark_times.items():
        if destination in calculated_times:
            calculated = calculated_times[destination]
            diff = abs(benchmark - calculated)
            diff_percentage = (diff / benchmark) * 100

            total_diff += diff
            total_diff_percentage += diff_percentage

            print(f"Destination {destination}:")
            print(f"  Benchmark Time: {benchmark} minutes")
            print(f"  Calculated Time: {calculated} minutes")
            print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
            print()
        else:
            print(f"Destination {destination} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all destinations.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")

In [12]:
intersection_penalty = 10
secret_scary_factor = 0.65
use_ambulance_speeds = False

od.set_graph_weights(intersection_penalty, secret_scary_factor, use_ambulance_speeds)

In [13]:
x, y = utils.geographic_to_utm(10.7343661,59.9369806)
start_node = get_node(x, y)

destinations = [(10.7740711,59.9276535), (10.7367191,59.9086137), (10.8131627,59.8967944)]

benchmark_times = {(10.7740711,59.9276535):7, # 6-8 minutes
                   (10.7367191,59.9086137):16, # 14-18 minutes
                   (10.8131627,59.8967944):14} # 14 minutter

In [14]:
calculated_times = {}

for destination in destinations:
    x, y = utils.geographic_to_utm(destination[0], destination[1])
    destination_node = get_node(x, y)

    shortest_time_path = ox.shortest_path(
                    od.graph,
                    start_node,
                    destination_node,
                    weight='time'
                )

    total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))
    
    calculated_times[destination] = total_travel_time
    
evaluate_travel_time_accuracy(benchmark_times, calculated_times)

Destination (10.7740711, 59.9276535):
  Benchmark Time: 7 minutes
  Calculated Time: 6.95308767179487 minutes
  Difference: 0.04691232820512958 minutes (0.67%)

Destination (10.7367191, 59.9086137):
  Benchmark Time: 16 minutes
  Calculated Time: 11.790558451282054 minutes
  Difference: 4.209441548717946 minutes (26.31%)

Destination (10.8131627, 59.8967944):
  Benchmark Time: 14 minutes
  Calculated Time: 13.458137878388275 minutes
  Difference: 0.5418621216117252 minutes (3.87%)

Total Discrepancy: 4.7982159985348005 minutes across all destinations.
Average Discrepancy Percentage: 10.28%
